# Week 6: A/B Testing I|
## AIM-5014-1A: Experimental Optimization
### David Sweet // 20230706

You are an ML engineer working on an ad-serving system. Users of your app are occasionally shown an ad. If the user clicks on the ad, they are given the option to make a purchase. If they do, you receive a payout from the selling. Your business metric is revenue/click, RPC.

Increasing RPC by at least \$0.02 would be valuable.

A colleague has prepared a new ad-ranker and would like to measure its impact on RPC via an A/B test. Let's call her new ad-ranker version B -- or *arm* B -- and the current, production ad-ranker version A -- or *arm* A. You'll be conducting the A/B test.

You've just learned about multi-armed bandits and Thompson Sampling and think it might improve the quality of your experiment. You'd like to give it a try. Since you've run many A/B tests, you already have working A/B testing code.  You would like to modify that code to take a measurement using Thompson Sampling for randomization instead of using simple A/B test-style (50/50) randomization.

The function `observe()`, below, emulates an observation, i.e. a single event where a user is shown an ad. `observe()` returns the revenue earned from showing the ad.

In [288]:
import numpy as np

def observe(arm):
    # User is shown an ad.
    if np.random.uniform() < .90:
        # If they don't click, then revenue is $0
        return 0
    
    # If they do click:
    if arm == "A":
        rpc = 1.00
    elif arm == "B":
        rpc = 1.75
    else:
        assert False, "arm should be A or B"
        
    # Return revenue for a single click, which is
    #  noisy, but equal to rpc in expectation.
    return rpc + .75*np.random.normal()

observe("A")

0

# Q1, 1 pt

The function `calculate_mean_and_se(y_arm)` takes a list, `y_arm`, of observations -- each item in the list being an output value of `observe()` -- and computes the mean and standard error.

Replace the word `TODO` with the code to calculate se.

In [289]:
def calculate_mean_and_se(y_arm):
    m = np.mean(y_arm)
    se = np.std(y_arm)/np.sqrt(len(y_arm))
    return m, se

# Q2, 1 pt
Recall that the average of $N$ iid draws from a random variable approximates a Gaussian distribution for large enough $N$. What statistics rule/law/theorem/concept tells us that this is the case?

**Answer:** Central Limit Theorem

In [290]:
def draw_from_gaussian_reward_model(y_arm):
    m, se = calculate_mean_and_se(y_arm)
    return m + se*np.random.normal()
# draw_from_gaussian_reward_model("A")

# Q3, 5 pts
The function `draw_from_gaussian_reward_model()` takes a draw from the Gaussian distribution that models the mean of `y_arm` (== the average RPC).

Write a function `thompson_sampling_randomizer(y_a, y_b)` that takes two lists of observations -- one for arm A and one for arm B -- then decides whether to show the user ad-ranker A or ad-ranker B for the next observation. Return your function's decision as the string "A" or the string "B". A function `ab_test_randomizer()` is provided for comparison.

In [291]:
def ab_test_randomizer(y_a, y_b):
    # This function ignores the arguments y_a and y_b
    #  and just chooses randomly, as is appropriate
    #  for normal A/B testing.
    if np.random.uniform() < 0.5:
        return "A"
    return "B"

In [313]:
def thompson_sampling_randomizer(y_a, y_b):
    # y_a, y_b are lists of observations, one for arm A and one for arm B
    # Choose an arm to use for the next observation via Thompson Sampling
    # Return the name of the chosen arm, either "A" or "B"
    # Hint: Make use of draw_from_gaussian_reward_model()
    tsa = draw_from_gaussian_reward_model(y_a)
    tsb = draw_from_gaussian_reward_model(y_b)

    if len(y_a)<10 or len(y_b)<10:
        return np.random.choice(["A", "B"])
    elif tsa > tsb:
        return "A"
    elif tsa == tsb:
        return np.random.choice(["A", "B"])
    return "B"
    

# Q4, 1 pt

Replace the word `TODO` (two places) in the function `design()` with the correct code.

In [314]:
def design():
    practical_significance = 0.02
    # pilot study
    sd_a = np.std([observe("A") for _ in range(100)])
    sd_delta = np.sqrt(2*sd_a**2)
    
    num_observations = int((2.5 * sd_delta / practical_significance) ** 2)
    
    return num_observations

# Q5, 2 pts

The function `measure()` emulates the process of taking a measurement. Note that the second argument, `randomizer_function`, is a Python function. It could be `ab_test_randomizer` or `thompson_sampling_randomizer`.

In [317]:
def measure(num_observations, randomizer_function):
    business_metric = []
    y_a = []
    y_b = []
    for _ in range(num_observations):             
        arm = randomizer_function(y_a, y_b)
        bm = observe(arm)
        if arm == "A":
            y_a.append(bm)
        else: # B
            y_b.append(bm)
        business_metric.append(bm)
        # if randomizer_function.__name__ == "thompson_sampling_randomizer":
        #     if len(y_b) >= 10 and len(y_a) >= 10:
                    
        #         if len(y_b) / len(y_a) >= 0.98 or len(y_a) / len(y_b) >= 0.98:
        #             break
        
    bm = np.mean(business_metric)
    print (f"num = {len(business_metric)} num_A = {len(y_a)} num_B = {len(y_b)}")
    print (f"bm = {bm:.4f} bm_A = {np.mean(y_a):.4f} bm_B = {np.mean(y_b):.4f}")

The code in the cell below will run `measure()` once with `ab_test_randomizer` and once with `thompson_sampling_randomizer`.

Run the cell a few times.  What is your interpretation of its output? Comment on each printed quantity.

In [318]:
import matplotlib.pyplot as plt
from time import time

np.random.seed(17)
num_observations = design()

start = time()
print ("AB_test_randomizer")
measure(num_observations, ab_test_randomizer)
ab_time = time() - start
print("Time for A/B test:", ab_time )

start = time()
print ("\nThompson_sampling_randomizer")
measure(num_observations, thompson_sampling_randomizer)
ts_time = time() - start
print("Time for MAB using Thompson sampling:", ts_time)

print(f"\nA/B test is {int(ts_time/ab_time) } times faster than Thompson sampling")

AB_test_randomizer
num = 3359 num_A = 1676 num_B = 1683
bm = 0.1399 bm_A = 0.1166 bm_B = 0.1632
Time for A/B test: 0.03477644920349121

Thompson_sampling_randomizer
num = 3359 num_A = 159 num_B = 3200
bm = 0.1501 bm_A = 0.0766 bm_B = 0.1537
Time for MAB using Thompson sampling: 4.044288635253906

A/B test is 116 times faster than Thompson sampling
